#### Parse data

In [2]:
!scrapy startproject films

New Scrapy project 'films', using template directory 'C:\Users\PC\anaconda3\Lib\site-packages\scrapy\templates\project', created in:
    D:\Inno\Course3\DWV\films

You can start your first spider with:
    cd films
    scrapy genspider example example.com


In [5]:
import scrapy
import pandas as pd

class TableSpider(scrapy.Spider):
    name = 'table_spider'
    start_urls = ['https://en.wikipedia.org/wiki/List_of_highest-grossing_films']
    
    def parse(self, response):
        h2 = response.xpath('//div[@class="mw-heading mw-heading2"]')[0]
        table = h2.xpath('following-sibling::table')[0]
        rows = table.xpath('.//tbody/tr')

        data=[]
        for i in range(len(rows)):
            row=rows[i]
            data_from_table = row.xpath('.//td | .//th')  # Select all cells (td for data, th for headers)
            data_from_table = [cell.xpath('normalize-space(.)').get() for cell in data_from_table]
            data.append(data_from_table)
    
        columns = ["Rank", "Peak", "Film Title", "Worldwide gross", "Release Year", "Ref"]
        df = pd.DataFrame(data, columns=columns)
        df.to_csv('table.csv', index=False)
        self.logger.info(f"Extracted DataFrame:\n{df}")
        
        # From obtained links, parse into separate df
        links = rows.xpath('.//th[@scope="row"]/i/a/@href').getall()
        for link in links:
            # go to link, parse data
            link = response.urljoin('https://en.wikipedia.org'+link)  # Construct full URL
            yield scrapy.Request(url=link, callback=self.parse_film_page)
            # data.append(box_data)
        
    def parse_film_page(self, response):
        box = response.xpath('//div[@class="mw-content-ltr mw-parser-output"]')[0]
        table = box.xpath('.//table[@class="infobox vevent"]')[0]
        rows = table.xpath('.//tbody/tr')
        
        info = {}
        target_rows = ["Directed by", "Countries", "Box office", "Country"]
        for row in rows:
            header = row.xpath('.//th[@scope="row"]/text()').get()
            if header in target_rows:
                value = row.xpath('.//td//text()').get()
                # Clean the value: join text, strip whitespace, and remove empty strings
                value = ''.join([text.strip() for text in value if text.strip()])
                info[header]=value
        
        df_infobox = pd.DataFrame([info])
        # Append to csv
        df_infobox.to_csv('box.csv', mode='a', header=False, index=False)
        self.logger.info(f"Extracted infobox data for {response.url}")

In [6]:
!scrapy runspider spider.py

2025-02-27 22:49:52 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-02-27 22:49:52 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 25.0.0 (OpenSSL 3.4.1 11 Feb 2025), cryptography 44.0.1, Platform Windows-10-10.0.26100-SP0
2025-02-27 22:49:53 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-27 22:49:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2025-02-27 22:49:53 [scrapy.extensions.telnet] INFO: Telnet Password: 2252fb10835134b3
2025-02-27 22:49:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2025-02-27 22:49:53 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2025-02-27 22:49:53

In [10]:
import  pandas as pd
table=pd.read_csv('table.csv')
box=pd.read_csv('box.csv',names=["Directed by", "Country", "Box office", "Name"])

# keys for appendings
box_key = box.iloc[:, 3]  
table_key = table['Film Title']

box['key'] = box_key
table['key'] = table_key

merged_df = pd.merge(table, box, on='key', how='inner')

merged_df = merged_df.drop(columns=['key', 'Peak', 'Ref', 'Name', 'Box office'])
merged_df['Worldwide gross']=merged_df['Worldwide gross'].apply(lambda x: ''.join( c for c in x if  c not in '$TSMF' )).apply(lambda x: ''.join(x.split(',')))

print(merged_df.loc[40])
merged_df.to_csv('data.csv')

Rank                            50
Film Title         Despicable Me 3
Worldwide gross         1034800131
Release Year                  2017
Directed by          Pierre Coffin
Country              United States
Name: 40, dtype: object


#### Load to MongoDB

In [11]:
import csv
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["films_db"]  
collection = db["films"] 

with open('./data.csv', 'r') as file:
    csv_data = csv.DictReader(file)  # Use DictReader to read rows as dictionaries

    for row in csv_data:
        # Convert numeric fields to integers
        row["Rank"] = int(row["Rank"])
        row["Worldwide gross"] = int(row["Worldwide gross"])
        row["Release Year"] = int(row["Release Year"])

        collection.insert_one(row)

print("Data inserted")

Data inserted


In [16]:
from bson import ObjectId
import json

# Fetch data from MongoDB collection
res = collection.find()

# Convert the MongoDB cursor to a list of dictionaries and remove unwanted fields
data = []
for doc in res:
    # Remove the '_id' and empty key ('') fields
    doc.pop('_id', None)  # Remove '_id' if it exists
    doc.pop('', None)     # Remove empty key if it exists
    data.append(doc)

# Write the data to a JSON file as a valid JSON array
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)  # Use json.dump directly with the list